# expand our data using scrapy
    - first, get the all unique course id
    - second,get the url and use `urlopen` to open it
    - third,scrapy

In [1]:
import codecs
import re
from pprint import pprint

## Get the URLs list for scrapy
1. get the unique coursename and store it into the dictionary

In [2]:
def get_unique_cname(filepath):
    cnameDic={}
    f=codecs.open(filepath,"r","utf-8")
    lines=f.readlines()[1:] 
#     doesn't read the header line
#     for i,j in enumerate(lines):
#         if i<10:
#             cname=re.split(",",j)[0].strip()
#             cnameDic[cname]=cnameDic.get(cname,0)+1
    for i in lines:
        cname=re.split(",",i)[0].strip()
        cnameDic[cname]=cnameDic.get(cname,0)+1
    f.close()
    return cnameDic

In [13]:
cnameDic=get_unique_cname(r"/Users/shuo/Documents/reviews_by_course.csv")
pprint(cnameDic)

{'2-speed-it': 25,
 '20cnwm': 2,
 '2d-cad': 25,
 '3d-cad': 19,
 '3d-printing': 10,
 '3d-printing-applications': 47,
 '3d-printing-revolution': 118,
 '3d-printing-software': 19,
 'a-programar': 270,
 'abdomen-anatomy': 50,
 'ableton-live': 188,
 'aboriginal-education': 6,
 'academic-discussion-english': 3,
 'academic-literacy': 11,
 'academic-writing-capstone': 11,
 'academicinfoseek': 25,
 'accounting-analytics': 172,
 'accounting-finance': 3,
 'accounting-for-managers': 24,
 'activism-social-movements': 2,
 'activismo-movimientos-sociales': 1,
 'addiction-and-the-brain': 214,
 'adhd-treatment': 44,
 'adjective-clauses': 37,
 'administracion-de-proyectos-capstone': 16,
 'administracion-estrategica': 37,
 'admonfinanciera': 34,
 'advanced-algorithms-and-complexity': 13,
 'advanced-calculus': 191,
 'advanced-chemistry': 59,
 'advanced-competitive-strategy': 74,
 'advanced-competitive-strategy-zh': 16,
 'advanced-data-structures': 149,
 'advanced-excel': 37,
 'advanced-manufacturing-enter

 'musicianship-harmony': 16,
 'muslim-world': 115,
 'nand2tetris2': 7,
 'nanotechnology1': 25,
 'nanotechnology2': 2,
 'natural-attenuation-of-groundwater-contaminants': 13,
 'natural-language-processing': 28,
 'nauchno-texnicheski-text': 4,
 'negociacao': 103,
 'negociacion': 153,
 'negociacion-proceso': 9,
 'negocios-internacionales': 83,
 'negocios-internacionales-2': 17,
 'negotiation': 192,
 'negotiation-fundamentals': 1,
 'negotiation-skills': 835,
 'network-biology': 10,
 'networks-illustrated': 6,
 'neural-networks': 246,
 'neurobiology': 46,
 'neuroeconomics': 129,
 'neurohacking': 10,
 'neuromarketing': 67,
 'new-nordic-diet': 31,
 'new-ventures': 1,
 'news-literacy': 8,
 'nonprofit-gov-2': 5,
 'nonprofit-gov-3': 1,
 'nonprofit-organizations': 18,
 'nonviolence': 27,
 'norms': 48,
 'note-taking': 15,
 'noun-clauses-conditionals': 10,
 'nsu-genetics': 31,
 'nsu-virology': 15,
 'nube-ios': 15,
 'nurture-market-strategies': 81,
 'nutrition-pregnancy': 7,
 'object-oriented-java':

2. get the url and write these urls to a file for next step
    
    `urlopen` to open it

In [14]:
domain="https://www.coursera.org/learn/"
# this is the first part of the url
# later, i will add the coursename to get the full url

In [15]:
def write_urllist(cnameDic,domain):
    for cname in cnameDic.keys():
        codecs.open("url_list.txt","a+",'utf-8').write(cname+"\t"+domain+cname+"\n")

In [16]:
# this line of code writes the url list to a txt file named "url_list.txt"
write_urllist(cnameDic,domain)

## Test each function on the whole URLs
3. test the url_list to scrapy the data 

In [2]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

In [81]:
def open_soup(url):
    try:
        html=urlopen(url).read().decode('utf-8')
        return BeautifulSoup(html, features='lxml')
    except HTTPError:
        print("\nthere is an 404 error in this url: ",url)
        return False     

def get_course_name(soup):
    return soup.find('h1',class_="H2_1pmnvep-o_O-weightNormal_s9jwp5-o_O-fontHeadline_1uu0gyz max-text-width-xl m-b-1s").contents[0]

def get_uni_name(soup):
    uni_list=[]
    uni_name_tag=soup.find("div",class_="partnerBanner_10e5gws-o_O-Box_120drhm-o_O-displayflex_poyjc")
    for i in uni_name_tag.contents[1:]:
        try:
            uni_name=i.contents[0]["title"]
        except KeyError:
            uni_name=i.contents[0].contents[0]
        uni_list.append(uni_name)
    return uni_list

def get_instructors_list(soup):
    # because there maybe more than one instructors 
    # instructors_dic key=name, value="url for the instructor"
    instructors_dic={}
    instructorList=soup.find_all('h3',class_="H2_1pmnvep-o_O-weightBold_uvlhiv-o_O-bold_1byw3y2 font-lg")
    for i in instructorList:
#         instructors_dic[i.contents[0].contents[0]]=i.contents[0]["href"]  #this is used to get the href of the instructor
        instructors_dic[i.contents[0].contents[0]]=get_clist_instr("https://www.coursera.org"+i.contents[0]["href"])
#         print(i.contents[0].contents[0])
#         print("here you can jump into the new website for the lecture page")
    return instructors_dic

def get_clist_instr(url):
    clist=[]
    instr_soup=BeautifulSoup(urlopen(url).read().decode('utf-8'), features='lxml')
    clist_tag=instr_soup.find_all("h4",class_="H4_1k76nzj-o_O-weightNormal_s9jwp5-o_O-fontBody_56f0wi")
    for i in clist_tag:
        clist.append(i.contents[0].contents[0].contents[0])
    return clist

def get_category(soup):
    cate=soup.find_all("div",class_="BreadcrumbItem_1pp1zxi")
    category={}
    for i,j in enumerate(cate):
        if i==0:
            continue
        category[i]=j.contents[0].contents[0]
    return category

def get_financial_aid(soup):
    return str(soup).find("Yes, Coursera provides")

def get_hours_lan(soup):
    contents=soup.find_all("h4",class_="H4_1k76nzj-o_O-weightBold_uvlhiv-o_O-bold_1byw3y2 m-b-0")
    hours_str=contents[-2].contents[0].contents[0]
    hours=re.split(" ",hours_str)[1]
    lan=contents[-1].contents[0]
    return hours,lan
def get_captions(soup):
    captions=soup.find_all("div",class_="Box_120drhm-o_O-displayflex_poyjc-o_O-columnDirection_ia4371")[-1].contents[1].contents
    return captions[0].contents[0]

def get_rates(soup):
    return soup.find("span",class_="H4_1k76nzj-o_O-weightBold_uvlhiv-o_O-bold_1byw3y2 m-l-1s m-r-1 m-b-0").contents[0]    

def get_num_rates_reviews(soup):
    num_rates_str=soup.find("div",class_="P_gjs17i-o_O-weightNormal_s9jwp5-o_O-fontBody_56f0wi m-r-1s")
    num_rates_comma=re.split(" ",num_rates_str.contents[0].contents[0])[0]
    num_reviews_str=soup.find("div",class_="P_gjs17i-o_O-weightNormal_s9jwp5-o_O-fontBody_56f0wi m-l-1s m-r-1")
    num_reviews_comma=re.split(" ",num_reviews_str.contents[0].contents[0])[0]
    return int("".join(re.split(",",num_rates_comma))),int("".join(re.split(",",num_reviews_comma)))

def get_skills_list(soup):
    skills_list=[]
    skills_tag=soup.find_all('span',class_="Pill_56iw91 m-r-1s m-b-1s")
    for i in skills_tag:
        skills_list.append(i.contents[0].contents[0].contents[0])
    return skills_list
#     need to do the further testing 

**test: get the course name function**

In [12]:
# test: get the course name function
def scrapy_data(url_list_file):
    cnameList=[] #to store the list of coursename
    f=codecs.open(url_list_file,"r","utf-8")
    for i,j in enumerate(f.readlines()):
        temp_url=re.split("\t",j)[1]
        temp_soup= open_soup(temp_url)   # a defined function
        if not temp_soup:
            continue 
        temp_cname=get_course_name(temp_soup)  #another define function
        cnameList.append(temp_cname)
        print(i,'\n'+temp_cname)
    return cnameList
# Feedback:
# there are some urls that will return 404 error


In [15]:
scrapy_data("url_list.txt")

0 
Two Speed IT: How Companies Can Surf the Digital Wave, a BCG Perspective
1 
20世纪西方音乐 Western Music in the 20th Century
2 
工程圖學 2D CAD
3 
工程圖學 3D CAD
4 
3D-печать для всех и каждого
5 
3D Printing Applications
6 
The 3D Printing Revolution
7 
3D Printing Software
8 
¡A Programar! Una introducción a la programación
9 
Anatomy of the Abdomen and Pelvis; a journey from basis to clinic.  
10 
Introduction to Ableton Live
11 
Aboriginal Worldviews and Education
12 
Academic Discussions in English
13 
Academic Literacy
14 
Project: Writing a Research Paper
15 
Academic Information Seeking
16 
Accounting Analytics
17 
Accounting and Finance for IT professionals
18 
Managerial Accounting: Cost Behaviors, Systems, and Analysis
19 
ART of the MOOC: Activism and Social Movements
20 
ART of the MOOC: Activismo y Movimientos Sociales
21 
The Addicted Brain
22 
ADHD: Everyday Strategies for Elementary Students
23 
Adjectives and Adjective Clauses
24 
Administración de Proyectos Project
25 
Adminis

KeyboardInterrupt: 

**test: get the category of course function**

In [16]:
# Test: get the category function
def scrapy_category_data(url_list_file):
    categoryList=[]
    f=codecs.open(url_list_file,"r","utf-8")
    for i,j in enumerate(f.readlines()):
        temp_url=re.split("\t",j)[1]
        temp_soup= open_soup(temp_url)   # a defined function
        if not temp_soup:
            continue 
        temp_category=get_category(temp_soup)
        categoryList.append(temp_category)
        print(i,'\n',temp_category)
    return categoryList

In [17]:
scrapy_category_data("url_list.txt")

0 
 {1: 'Business', 2: 'Business Essentials'}
1 
 {1: 'Arts and Humanities', 2: 'Music and Art'}
2 
 {1: 'Physical Science and Engineering', 2: 'Mechanical Engineering'}
3 
 {1: 'Physical Science and Engineering', 2: 'Mechanical Engineering'}
4 
 {1: 'Computer Science', 2: 'Design and Product'}
5 
 {1: 'Business', 2: 'Entrepreneurship'}
6 
 {1: 'Business', 2: 'Entrepreneurship'}
7 
 {1: 'Business', 2: 'Entrepreneurship'}
8 
 {1: 'Computer Science', 2: 'Software Development'}
9 
 {1: 'Life Sciences', 2: 'Medicine and Healthcare'}
10 
 {1: 'Arts and Humanities', 2: 'Music and Art'}
11 
 {1: 'Social Sciences', 2: 'Education'}
12 
 {1: 'Language Learning', 2: 'Learning English'}
13 
 {1: 'Language Learning', 2: 'Learning English'}
14 
 {1: 'Language Learning', 2: 'Learning English'}
15 
 {1: 'Personal Development', 2: 'Personal Development'}
16 
 {1: 'Business', 2: 'Finance'}
17 
 {1: 'Business', 2: 'Finance'}
18 
 {1: 'Business', 2: 'Finance'}
19 
 {1: 'Arts and Humanities', 2: 'Music and

KeyboardInterrupt: 

**test:the get list of instructors function**

In [18]:
# test: to get the instructors list
# add the function, making the function to get the courses list of the instructor
def scrapy_instructors_data(url_list_file):
    instructorsList=[]
    f=codecs.open(url_list_file,"r","utf-8")
    for i,j in enumerate(f.readlines()):
        temp_url=re.split("\t",j)[1]
        temp_soup= open_soup(temp_url)   # a defined function
        if not temp_soup:
            continue 
        temp_instructors=get_instructors_list(temp_soup)
        instructorsList.append(temp_instructors)
        print(i,'\n',temp_instructors)
    return instructorsList


In [82]:
scrapy_instructors_data("url_list.txt")

0 
 {'Antoine Gourévitch': ['Two Speed IT: How Companies Can Surf the Digital Wave, a BCG Perspective'], 'Vanessa Lyon': ['Two Speed IT: How Companies Can Surf the Digital Wave, a BCG Perspective'], 'Eric Baudson': ['Two Speed IT: How Companies Can Surf the Digital Wave, a BCG Perspective']}
1 
 {'毕明辉 Bryan Minghui BI': ['20世纪西方音乐 Western Music in the 20th Century']}
2 
 {'康仕仲': ['工程圖學 2D CAD', '工程圖學 2D CAD 專題', '3D CAD Application', '3D CAD Fundamental', '工程圖學 3D CAD 專題', '工程圖學 3D CAD']}
3 
 {'康仕仲': ['工程圖學 2D CAD', '工程圖學 2D CAD 專題', '3D CAD Application', '3D CAD Fundamental', '工程圖學 3D CAD 專題', '工程圖學 3D CAD']}
4 
 {'Николай Георгиевич Булахов (Nikolay G. Bulakhov)': ['Сетевое администрирование: от теории к практике', '3D-печать для всех и каждого']}
5 
 {'Vishal Sachdev': ['Digital Marketing Capstone', '3D Printing Applications']}
6 
 {'Aric Rindfleisch': ['Marketing in a Digital World', 'Digital Marketing Capstone', 'Marketing in an Analog World', 'The 3D Printing Revolution']}
7 
 {'

KeyboardInterrupt: 

**test: get the university name**

In [39]:
# to get the uniname list 
def scrapy_uni_data(url_list_file):
    uninameList=[]
    f=codecs.open(url_list_file,"r","utf-8")
    for i,j in enumerate(f.readlines()):
        temp_url=re.split("\t",j)[1]
        temp_soup= open_soup(temp_url)   # a defined function
        if not temp_soup:
            continue 
        temp_uniName=get_uni_name(temp_soup)
        uninameList.append(temp_uniName)
        print(i,'\n',temp_uniName)
    return uninameList
# feedback:
# it maybe plain text, not image 
# so i use the try . exception to get the uniname of image and text


In [37]:
scrapy_uni_data("url_list.txt")

0 
 ['CentraleSupélec']
1 
 ['Peking University']
2 
 ['National Taiwan University']
3 
 ['National Taiwan University']
4 
 ['National Research Tomsk State University']
5 
 ['University of Illinois at Urbana-Champaign']
6 
 ['University of Illinois at Urbana-Champaign']
7 
 ['University of Illinois at Urbana-Champaign']
8 
 ['Universidad ORT Uruguay', 'The University of Edinburgh']
9 
 ['Universiteit Leiden', 'Leiden University Medical Center']
10 
 ['Berklee College of Music']
11 
 ['University of Toronto']
12 
 ['University of California, Irvine']
13 
 ['Moscow Institute of Physics and Technology']
14 
 ['University of California, Irvine']
15 
 ['University of Copenhagen', 'Technical University of Denmark (DTU)']
16 
 ['University of Pennsylvania']
17 
 ['Indian School of Business']
18 
 ['University of Illinois at Urbana-Champaign']
19 
 ['Duke University', 'Creative Time ']
20 
 ['Duke University', 'Creative Time ']
21 
 ['Emory University']
22 
 ['The State University of New York'

KeyboardInterrupt: 

**test:whether there is a financial aid**

In [42]:
# to get whether the course is financial aid 
def scrapy_financial_aid(url_list_file):
    financialList=[]
    f=codecs.open(url_list_file,"r","utf-8")
    for i,j in enumerate(f.readlines()):
        temp_url=re.split("\t",j)[1]
        temp_soup= open_soup(temp_url)   # a defined function
        if not temp_soup:
            continue 
        if get_financial_aid(temp_soup)>-1:
            temp_financialaid=True  #if the index is not -1 then return True, meaning the financial aid is available
        else:
            temp_financialaid=False 
        print(i,'\n',temp_financialaid)
        financialList.append(temp_financialaid)
    return financialList


In [43]:
scrapy_financial_aid("url_list.txt")

0 
 True
1 
 True
2 
 True
3 
 True
4 
 True
5 
 True
6 
 True
7 
 True
8 
 True
9 
 True
10 
 True
11 
 True
12 
 True
13 
 True
14 
 True
15 
 True
16 
 True
17 
 True
18 
 True
19 
 True
20 
 True
21 
 True
22 
 True
23 
 True
24 
 True
25 
 True
26 
 True
27 
 True

there is an 404 error in this url:  https://www.coursera.org/learn/advanced-calculus

29 
 False
30 
 True
31 
 False
32 
 True
33 
 True
34 
 True
35 
 True
36 
 True


KeyboardInterrupt: 

**test:get the hours and language of the course**

In [49]:
# to get the hours and the language 
def scrapy_hours_language_data(url_list_file):
    languageList=[]
    f=codecs.open(url_list_file,"r","utf-8")
    for i,j in enumerate(f.readlines()):
        temp_url=re.split("\t",j)[1]
        temp_soup= open_soup(temp_url)   # a defined function
        if not temp_soup:
            continue        
        temp_hours,temp_language=get_hours_lan(temp_soup)
#         languageList.append(temp_language)
        print(i,'\n',temp_hours,temp_language)
    return languageList


In [50]:
scrapy_hours_language_data("url_list.txt")

0 
 Approx. 22 hours to complete English
1 
 Approx. 17 hours to complete Chinese (Simplified)


KeyboardInterrupt: 

**test: get the captions**

In [54]:
# get the captions 
def scrapy_captions_data(url_list_file):
    captionsList=[]
    f=codecs.open(url_list_file,"r","utf-8")
    for i,j in enumerate(f.readlines()):
        temp_url=re.split("\t",j)[1]
        temp_soup= open_soup(temp_url)   # a defined function
        if not temp_soup:
            continue        
        temp_captions=get_captions(temp_soup)
        print(i,'\n',temp_captions)
    return captionsList


In [66]:
scrapy_captions_data("url_list.txt")

0 
 Subtitles: English
1 
 Subtitles: Chinese (Simplified)
2 
 Subtitles: Chinese (Traditional)
3 
 Subtitles: Chinese (Traditional)
4 
 Subtitles: Russian
5 
 Subtitles: English, French
6 
 Subtitles: English, Chinese (Simplified)
7 
 Subtitles: English, Russian
8 
 Subtitles: Spanish
9 
 Subtitles: English, Russian
10 
 Subtitles: English, Chinese (Simplified)
11 
 Subtitles: English
12 
 Subtitles: English, Albanian
13 
 Subtitles: English
14 
 Subtitles: English, Hindi
15 
 Subtitles: English, Portuguese (Brazilian)
16 
 Subtitles: English, Mongolian
17 
 Subtitles: English
18 
 Subtitles: English, Vietnamese
19 
 Subtitles: English, Spanish
20 
 Subtitles: Spanish
21 
 Subtitles: English, Greek, Chinese (Simplified)
22 
 Subtitles: English, Spanish
23 
 Subtitles: English, Spanish, Hindi
24 
 Subtitles: Spanish
25 
 Subtitles: Spanish
26 
 Subtitles: Spanish
27 
 Subtitles: English

there is an 404 error in this url:  https://www.coursera.org/learn/advanced-calculus

29 
 Subtitle

KeyboardInterrupt: 

**test: get the rates, number of the rates, num of the reviews of the course**

In [70]:
# to get the num of the rates and the reviews of the course 
def scrapy_num_rates_reviews_data(url_list_file):
    ratesList=[]
    f=codecs.open(url_list_file,"r","utf-8")
    for i,j in enumerate(f.readlines()):
        temp_url=re.split("\t",j)[1]
        temp_soup= open_soup(temp_url)   # a defined function
        if not temp_soup:
            continue        
        temp_num_rates,temp_num_reviews=get_num_rates_reviews(temp_soup)
#         ratesList.append(temp_rates)
        print(i,'\n',temp_num_rates,"\t",temp_num_reviews)
    return ratesList


In [71]:
scrapy_num_rates_reviews_data("url_list.txt")

0 
 128 	 33
1 
 23 	 4
2 
 192 	 30
3 
 91 	 23
4 
 105 	 15
5 
 339 	 85
6 
 681 	 211
7 
 187 	 55
8 
 962 	 424
9 
 308 	 90
10 
 788 	 240
11 
 50 	 25
12 
 25 	 6
13 
 93 	 20
14 
 68 	 18
15 
 157 	 46
16 
 1579 	 286
17 
 72 	 23
18 
 337 	 69
19 
 39 	 15
20 
 9 	 1
21 
 1181 	 291
22 
 616 	 185
23 
 362 	 99
24 
 70 	 26
25 
 367 	 140
26 
 311 	 125
27 
 234 	 54

there is an 404 error in this url:  https://www.coursera.org/learn/advanced-calculus

29 
 340 	 84
30 
 485 	 99
31 
 79 	 19
32 
 939 	 225
33 
 1354 	 198
34 
 85 	 19
35 
 124 	 25
36 
 50 	 11
37 
 244 	 67
38 
 338 	 86
39 
 160 	 18
40 
 596 	 132
41 
 55 	 17
42 
 34 	 11
43 
 254 	 83
44 
 1014 	 141


KeyboardInterrupt: 

**test: the function of the skill scrapy**

In [73]:
#to get the skills list 
def scrapy_skills_data(url_list_file):
    skillsList=[]
    f=codecs.open(url_list_file,"r","utf-8")
    for i,j in enumerate(f.readlines()):
        temp_url=re.split("\t",j)[1]
        temp_soup= open_soup(temp_url)   # a defined function
        if not temp_soup:
            continue        
        temp_skills=get_skills_list(temp_soup)
#         ratesList.append(temp_rates)
        print(i,'\n',temp_skills)
    return skillslist

# feedback:
# some of them don't have the list of skill

In [74]:
scrapy_skills_data("url_list.txt")

0 
 []
1 
 []
2 
 []
3 
 []
4 
 []
5 
 []
6 
 ['Materials', 'Product Development', 'New Product Development', 'Human–Computer Interaction']
7 
 []
8 
 ['Programming Tool', 'Software Development', 'Computer Programming', 'Scratch (Programming Language)']
9 
 []
10 
 ['Audio Recording', 'File Management', 'Music mixing', 'Abelton Live', 'MIDI programming']
11 
 []
12 
 []
13 
 []
14 
 []
15 
 []
16 
 ['Accounting', 'Analytics', 'Earnings Management', 'Finance']
17 
 []
18 
 ['Management Accounting', 'Cost Accounting', 'Activity Based Costing', 'Cost']
19 
 []
20 
 []
21 
 ['Prevention Programs', 'Addiction Treatment', 'Brain', 'Drug And Alcohol Abuse Treatment']
22 
 ['Teaching', 'Communication', 'Planning', 'Attention Deficit Hyperactivity Disorder (ADHD)']
23 
 ['Apposition', 'Grammar', 'English Language', 'Adjective']
24 
 []
25 
 ['Marketing', 'Innovation', 'Competitiveness', 'Product (Business)']
26 
 []
27 
 ['Python Programming', 'Linear Programming (LP)', 'Np-Completeness', 'Dyna

KeyboardInterrupt: 